In [275]:
import pandas as pd
import numpy as np
import os

In [276]:
from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [277]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Lambda
from keras.layers import Layer
from keras.layers import Softmax


import tensorflow as tf
import keras
import keras.backend as K

In [278]:
# Params
maxlen=5
embedding_dim = 100
text_vocabulary_size = 140000
n_epochs = 500

## Load data

In [279]:
w2v =Word2Vec.load('w2v.model')

### Real Events

In [360]:
# Turn extracted events into dataframe
line_list = []
o1_list = []
p_list = []
o2_list = []

# To change to training
with open('./train_events', 'r', encoding='utf8', errors='replace') as f:
    for event in f:
        event = event.split('\t')
        line_list.append(event[1])
        o1_list.append(event[2])
        p_list.append(event[3])
        o2_list.append(event[4])

event_ext_df = pd.DataFrame({'line_num': line_list, 'o1': o1_list, 'p': p_list, 'o2': o2_list})

### Fake o1

In [361]:
vocab = w2v.wv.vocab.keys()
vocab = list(vocab)
vocab_len = len(vocab)

In [362]:
o1_len = event_ext_df['o1'].str.split()

In [363]:
def get_random_key(vocab, vocab_len, length) :
    idx = np.random.randint(0, vocab_len, size=length)
    return [vocab[i] for i in idx]

In [364]:
fake_events = []
for length in o1_len.str.len():
    fake_events.append(get_random_key(vocab, vocab_len, length))
fake_events = [' '.join(list(x)) for x in fake_events]

In [365]:
event_ext_df['o1_fake'] = fake_events

In [ ]:
# all_events_df = pd.read_pickle('combined_event_df.pkl')
# all_events = all_events_df['o1']
# event_ext_df['o1_fake'] = all_events.sample(event_ext_df.shape[0]).values
# event_ext_df.head()

## Associate word_indices with embeddings

In [366]:
# Preprocess
event_ext_df['o1'] = event_ext_df.o1.map(simple_preprocess)
event_ext_df['p'] = event_ext_df.p.map(simple_preprocess)
event_ext_df['o2'] = event_ext_df.o2.map(simple_preprocess)
event_ext_df['o1_fake'] = event_ext_df.o1_fake.map(simple_preprocess)

In [367]:
# Tokenize and fit on text
t = Tokenizer()
t.fit_on_texts(event_ext_df['o1'] + event_ext_df['p'] + event_ext_df['o2'] + event_ext_df['o1_fake'])

In [368]:
# Text to sequence
event_ext_df['input_o1'] = t.texts_to_sequences(event_ext_df['o1'])
event_ext_df['input_p'] = t.texts_to_sequences(event_ext_df['p'])
event_ext_df['input_o2'] = t.texts_to_sequences(event_ext_df['o2'])
event_ext_df['input_o1_fake'] = t.texts_to_sequences(event_ext_df['o1_fake'])

In [369]:
# Pad sequences
input_o1 = pad_sequences(event_ext_df['input_o1'], maxlen=maxlen, padding='post')
input_p = pad_sequences(event_ext_df['input_p'], maxlen=maxlen, padding='post')
input_o2 = pad_sequences(event_ext_df['input_o2'], maxlen=maxlen, padding='post')
input_o1_fake = pad_sequences(event_ext_df['input_o1_fake'], maxlen=maxlen, padding='post')

In [370]:
event_ext_df['input_o1'] = [np.squeeze(x) for x in np.split(input_o1, input_o1.shape[0])]
event_ext_df['input_p'] = [np.squeeze(x) for x in np.split(input_p, input_o1.shape[0])]
event_ext_df['input_o2'] = [np.squeeze(x) for x in np.split(input_o2, input_o1.shape[0])]
event_ext_df['input_o1_fake'] = [np.squeeze(x) for x in np.split(input_o1_fake, input_o1.shape[0])]

### Create weight matrix

In [371]:
# From https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
embedding_matrix = np.zeros((text_vocabulary_size, 100))

In [372]:
#create a weight matrix for words in training docs
for word, i in t.word_index.items():
    try:
        embedding_vector = w2v.wv.get_vector(word)
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue

### Aggregate embeddings

In [373]:
event_ext_df['input_o1'] = [embedding_matrix[x] for x in event_ext_df['input_o1']]
event_ext_df['input_p'] = [embedding_matrix[x] for x in event_ext_df['input_p']]
event_ext_df['input_o2'] = [embedding_matrix[x] for x in event_ext_df['input_o2']]
event_ext_df['input_o1_fake'] = [embedding_matrix[x] for x in event_ext_df['input_o1_fake']]

In [374]:
g = event_ext_df.groupby('line_num')

In [375]:
agg_df = pd.concat([g.input_o1.apply(np.mean, axis=0),
                    g.input_p.apply(np.mean, axis=0),
                    g.input_o2.apply(np.mean, axis=0), 
                    g.input_o1_fake.apply(np.mean, axis=0)],
                   axis=1)

In [376]:
agg_df.reset_index(inplace=True)
agg_df['line_num'] = agg_df['line_num'].astype('int')
agg_df = agg_df.sort_values('line_num')
agg_df.reset_index(inplace=True, drop=True)

In [378]:
agg_df.to_pickle('agg_train_df.pkl')

## Load data

In [5]:
agg_df = pd.read_pickle('agg_train_df.pkl')

In [6]:
input_o1 = agg_df.input_o1.values
input_p = agg_df.input_p.values
input_o2 = agg_df.input_o2.values
input_o1_fake = agg_df.input_o1_fake.values

## Event Embedding Model

In [7]:
import scipy.stats as stats

class NeuralTensorLayer(Layer):
    # Credits tp dapurv5 at https://github.com/dapurv5/keras-neural-tensor-layer/
    def __init__(self, output_dim, input_dim, **kwargs):
        self.output_dim = output_dim #k
        self.input_dim = input_dim   #d
        if self.input_dim:
            kwargs['input_shape'] = (self.input_dim,)
        super(NeuralTensorLayer, self).__init__(**kwargs)


    def build(self, input_shape):
        mean = 0.0
        std = 1.0
        # W : k*d*d
        k = self.output_dim
        d = self.input_dim
        initial_T_values = stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(k,d,d))
        initial_V_values = stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(2*d,k))
        self.T = K.variable(initial_T_values)
        self.V = K.variable(initial_V_values)
        self.b = K.zeros((self.input_dim,))
        self.trainable_weights = [self.T, self.V, self.b]
        #self.trainable_weights = [self.T]


    def call(self, inputs, mask=None):
        if type(inputs) is not list or len(inputs) <= 1:
              raise Exception('BilinearTensorLayer must be called on a list of tensors '
                              '(at least 2). Got: ' + str(inputs))
        e1 = inputs[0]
        e2 = inputs[1]
        batch_size = K.shape(e1)[0]
        k = self.output_dim
        # print([e1,e2])
        feed_forward_product = K.dot(K.concatenate([e1,e2]), self.V)
        # print(feed_forward_product)
        bilinear_tensor_products = [ K.sum((e2 * K.dot(e1, self.T[0])) + self.b, axis=1) ]
        #bilinear_tensor_products = [ K.sum((e2 * K.dot(e1, self.T[0])), axis=1) ]
        # print(bilinear_tensor_products)
        for i in range(k)[1:]:
            btp = K.sum((e2 * K.dot(e1, self.T[i])) + self.b, axis=1)
            #btp = K.sum((e2 * K.dot(e1, self.T[i])), axis=1)
            bilinear_tensor_products.append(btp)
        result = K.tanh(K.reshape(K.concatenate(bilinear_tensor_products, axis=0), (batch_size, k)) + feed_forward_product)
        #result = K.reshape(K.concatenate(bilinear_tensor_products, axis=0), (batch_size, k))
        # print(result)
        return result


    def compute_output_shape(self, input_shape):
    # print (input_shape)
        batch_size = input_shape[0][0]
        return (batch_size, self.output_dim)

In [8]:
def l2_reg(weight_matrix):
    return 0.0001 * K.sum(K.square(weight_matrix))

In [233]:
K.clear_session()

In [ ]:
# Embed events
# Take training events and return embedded vectors
# Create copy of training events, randomly sample events from data, return embedded vectors
# Multiply each by tensor
# Calculate loss


# Take real inputs
input_o1_ = Input(shape=(maxlen, embedding_dim), dtype='float32', name='o1')
input_p_ = Input(shape=(maxlen, embedding_dim), dtype='float32', name='p')
input_o2_ = Input(shape=(maxlen, embedding_dim), dtype='float32', name='o2')
input_o1_fake_ = Input(shape=(maxlen, embedding_dim), dtype='float32', name='o1_fake')

# # # Get embeddings
# embedding_layer_shared = Embedding(text_vocabulary_size, embedding_dim)
# embedded_o1_ = embedding_layer_shared(input_o1_)
# embedded_p_ = embedding_layer_shared(input_p_)
# embedded_o2_ = embedding_layer_shared(input_o2_)

# # print(embedded_o1_)
# # Average embeddings for o1, p, o2
average = keras.layers.Lambda(lambda x: keras.backend.mean(x, axis=1))
o1_ = average(input_o1_) # Output dim (100)
p_ = average(input_p_)
o2_ = average(input_o2_)
o1_fake_ = average(input_o1_fake_)

# Create shared layers for real and fake embeddings
ntl_output_size = 50

ntl_r1 = NeuralTensorLayer(ntl_output_size, o1_.shape[1]) 
ntl_r2 = NeuralTensorLayer(ntl_output_size, p_.shape[1]) 
ntl_u = NeuralTensorLayer(ntl_output_size, ntl_output_size) 

# Real embeddings
r1_ = ntl_r1([o1_, p_])
r2_ = ntl_r2([p_, o2_])
u_ = ntl_u([r1_, r2_])

# Fake embeddings
r1_fake_ = ntl_r1([o1_fake_, p_])
u_fake_ = ntl_u([r1_fake_, r2_])

# Feed forward
feed_forward = Dense(1, activation='tanh', use_bias=True, bias_initializer='zeros')
u_out_ = feed_forward(u_)
u_fake_out_ = feed_forward(u_fake_)

# Instantiate model
# u = u_
# u_out = u_out_
# u_fake_out = u_fake_out_

embedder = Model(inputs=[input_o1_, input_p_, input_o2_, input_o1_fake_], 
                 outputs=[u_out_, u_fake_out_, u_])

loss =  K.maximum(0., 1 - u_out_ 
                        + u_fake_out_  
                        + l2_reg(embedder.layers[5].get_weights()[0]) 
                        + l2_reg(embedder.layers[6].get_weights()[0])
                        + l2_reg(embedder.layers[7].get_weights()[0])
                        + l2_reg(embedder.layers[8].get_weights()[0])
                        + l2_reg(embedder.layers[8].get_weights()[1]))
embedder.add_loss(loss)

In [ ]:
# tf.keras.backend.clear_session()
# Take event embeddings
event_ = Input(shape=(event_length, embedding_dim), dtype='int32', name='o1')
stock_ = Input(shape=(1,), dtype='int32')

# Create convolutional layers
# https://keras.io/layers/convolutional/

event_day_ = Conv1D(filters=embedding_dim, kernel_size=1, strides=1, padding='same', activation='relu',
                    use_bias=True, kernel_regularizer=l2_reg, bias_regularizer=l2_reg
                   )(event_)

event_week_ = Conv1D(filters=embedding_dim, kernel_size=7, strides=1, padding='same', activation='relu',
                     use_bias=True, kernel_regularizer=l2_reg, bias_regularizer=l2_reg
                    )(event_)

event_month_ = Conv1D(filters=embedding_dim, kernel_size=30, strides=1, padding='same', activation='relu',
                      use_bias=True, kernel_regularizer=l2_reg, bias_regularizer=l2_reg
                     )(event_)

# Max pooling of weekly and monthly events
# Narrow convolution of 3 neighbouring
max_pool_week_ = MaxPooling1D(pool_size=3, strides=None, padding='valid', data_format='channels_last'
                             )(event_week_)

max_pool_month_ = MaxPooling1D(pool_size=3, strides=None, padding='valid', data_format='channels_last'
                              )(event_month_)

# Concatenate daily, weekly, monthly
concat_ = keras.layers.Concatenate(axis=-1)([event_day_, event_week_, event_month_])

# Hidden layer
hidden_ = keras.layers.Dense(units=100, activation='relu', use_bias=True, kernel_initializer='glorot_uniform', 
                             bias_initializer='zeros', kernel_regularizer=l2_reg, bias_regularizer=l2_reg,
                            )(concat_)

# Softmax layer
output_ = keras.layers.Softmax(axis=-1)(hidden_)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model = Model(inputs=[event_, stock_], outputs=output_)

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), nb_epoch=10, batch_size=1024, verbose=2)
print(model.summary())

In [234]:
# Embed events
# Take training events and return embedded vectors
# Create copy of training events, randomly sample events from data, return embedded vectors
# Multiply each by tensor
# Calculate loss


# Take real inputs
input_o1_ = Input(shape=(maxlen, embedding_dim), dtype='float32', name='o1')
input_p_ = Input(shape=(maxlen, embedding_dim), dtype='float32', name='p')
input_o2_ = Input(shape=(maxlen, embedding_dim), dtype='float32', name='o2')
input_o1_fake_ = Input(shape=(maxlen, embedding_dim), dtype='float32', name='o1_fake')

# # # Get embeddings
# embedding_layer_shared = Embedding(text_vocabulary_size, embedding_dim)
# embedded_o1_ = embedding_layer_shared(input_o1_)
# embedded_p_ = embedding_layer_shared(input_p_)
# embedded_o2_ = embedding_layer_shared(input_o2_)

# # print(embedded_o1_)
# # Average embeddings for o1, p, o2
average = keras.layers.Lambda(lambda x: keras.backend.mean(x, axis=1))
o1_ = average(input_o1_) # Output dim (100)
p_ = average(input_p_)
o2_ = average(input_o2_)
o1_fake_ = average(input_o1_fake_)

# Create shared layers for real and fake embeddings
ntl_output_size = 50

ntl_r1 = NeuralTensorLayer(ntl_output_size, o1_.shape[1]) 
ntl_r2 = NeuralTensorLayer(ntl_output_size, p_.shape[1]) 
ntl_u = NeuralTensorLayer(ntl_output_size, ntl_output_size) 

# Real embeddings
r1_ = ntl_r1([o1_, p_])
r2_ = ntl_r2([p_, o2_])
u_ = ntl_u([r1_, r2_])

# Fake embeddings
r1_fake_ = ntl_r1([o1_fake_, p_])
u_fake_ = ntl_u([r1_fake_, r2_])

# Feed forward
feed_forward = Dense(1, activation='tanh', use_bias=True, bias_initializer='zeros')
u_out_ = feed_forward(u_)
u_fake_out_ = feed_forward(u_fake_)
 
embedder = Model(inputs=[input_o1_, input_p_, input_o2_, input_o1_fake_], 
                 outputs=[u_, u_out_, u_fake_out_])



In [235]:
loss =  K.maximum(0.,
                  1 
                  - u_out_ 
                  + u_fake_out_
                  + l2_reg(embedder.layers[5].get_weights()[0]) 
                  + l2_reg(embedder.layers[5].get_weights()[1])
                  + l2_reg(embedder.layers[5].get_weights()[2])
                  + l2_reg(embedder.layers[6].get_weights()[0]) 
                  + l2_reg(embedder.layers[6].get_weights()[1])
                  + l2_reg(embedder.layers[6].get_weights()[2])
                  + l2_reg(embedder.layers[7].get_weights()[0]) 
                  + l2_reg(embedder.layers[7].get_weights()[1])
                  + l2_reg(embedder.layers[7].get_weights()[2])
                  + l2_reg(embedder.layers[8].get_weights()[0])
                  + l2_reg(embedder.layers[8].get_weights()[1])
                 )
embedder.add_loss(loss)

In [236]:
embedder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
o1 (InputLayer)                 (None, 5, 100)       0                                            
__________________________________________________________________________________________________
p (InputLayer)                  (None, 5, 100)       0                                            
__________________________________________________________________________________________________
o2 (InputLayer)                 (None, 5, 100)       0                                            
__________________________________________________________________________________________________
o1_fake (InputLayer)            (None, 5, 100)       0                                            
__________________________________________________________________________________________________
lambda_1 (

In [237]:
adam = keras.optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [238]:
embedder.compile(optimizer=adam)

In [184]:
df = agg_df

In [240]:
test = embedder.fit(
            x=[K.expand_dims(df.iloc[0]['input_o1'], axis=0),
               K.expand_dims(df.iloc[0]['input_p'], axis=0),
               K.expand_dims(df.iloc[0]['input_o2'], axis=0),
               K.expand_dims(df.iloc[0]['input_o1_fake'], axis=0)],
            shuffle=False,
            steps_per_epoch=1
        )

Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 90.0029


In [241]:
embedder.predict(
            x=[K.expand_dims(df.iloc[1]['input_o1'], axis=0),
               K.expand_dims(df.iloc[1]['input_p'], axis=0),
               K.expand_dims(df.iloc[1]['input_o2'], axis=0),
               K.expand_dims(df.iloc[1]['input_o1_fake'], axis=0)],
            steps=1
        )

[array([[-0.9999999 , -0.0306939 ,  1.        , -1.        , -0.99794966,
         -0.97432613, -0.9995784 ,  0.9999998 , -0.9982622 , -1.        ,
         -1.        ,  0.79852283, -0.99999994, -0.9641454 ,  0.5456989 ,
          0.449236  , -1.        ,  0.9999999 , -0.999999  ,  1.        ,
          1.        ,  0.99990237, -1.        ,  0.99999976,  1.        ,
         -0.9955319 , -1.        , -0.96706736,  0.9999995 , -1.        ,
         -1.        ,  1.        , -0.7957741 ,  1.        ,  0.9886942 ,
         -0.8227973 ,  1.        , -0.8571624 , -1.        , -1.        ,
         -1.        , -1.        , -1.        ,  1.        , -1.        ,
         -1.        ,  0.9430496 , -1.        , -1.        , -0.62547123]],
       dtype=float32),
 array([[-0.93338436]], dtype=float32),
 array([[0.95585084]], dtype=float32)]

In [27]:
embedder.fit(
            x=[K.expand_dims(df.iloc[1]['input_o1'], axis=0),
               K.expand_dims(df.iloc[1]['input_p'], axis=0),
               K.expand_dims(df.iloc[1]['input_o2'], axis=0),
               K.expand_dims(df.iloc[1]['input_o1_fake'], axis=0)],
            y=0,
            shuffle=False,
            steps_per_epoch=1
        )

Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 88.0632


In [244]:
# Embedding loop
u_list = []
for i in df.index:
    for epoch in range(10):
        #if loss > 0:
        x = [K.expand_dims(df.iloc[i]['input_o1'], axis=0),
             K.expand_dims(df.iloc[i]['input_p'], axis=0),
             K.expand_dims(df.iloc[i]['input_o2'], axis=0),
             K.expand_dims(df.iloc[i]['input_o1_fake'], axis=0)]
        
        embedder.fit(
            x=x,
            y=None,
            shuffle=False,
            steps_per_epoch=1
        )
    result = embedder.predict(
        x=x,
        steps=1)
    print(result)
    
    u_list.append(result[0])
        
#         else:
#             break


Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 89.9570
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 89.8887
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 89.8745
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 89.7387
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 89.1501
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 88.8602
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 88.9155
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 88.6496
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 88.5825
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 88.5134
[array([[-1.        , -1.        , -1.        ,  1.        , -1.        ,
         1.        ,  0.99999905, -1.        , -1.        , -1.        ,
        -1.        ,  1.        , -1.        ,  1.        ,  1.        ,
         1.        ,

1/1 [==============================] - 8s 8s/step - loss: 88.6123
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.1870
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.0648
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.0828
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.0739
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.0379
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.0384
[array([[-0.9999989 , -1.        , -1.        ,  1.        ,  1.        ,
        -1.        , -1.        ,  0.9187182 , -1.        , -1.        ,
        -1.        ,  1.        , -1.        , -1.        ,  1.        ,
         1.        , -1.        ,  1.        , -1.        ,  1.        ,
         1.        ,  1.        , -1.        , -1.        , -1.        ,
        -1.        , -0.99999636,  1.        , -1.        ,  1.        ,
        -1.        ,  1.        , -1.  

Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 89.7356
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 89.6572
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.9711
[array([[-1.        , -1.        ,  1.        ,  1.        ,  1.        ,
         1.        , -1.        , -0.4450502 , -0.9999997 , -0.99999946,
        -1.        ,  1.        , -1.        ,  1.        , -0.938349  ,
        -1.        ,  0.99179775, -1.        , -1.        ,  0.98724407,
        -1.        , -0.62230426,  1.        ,  1.        ,  1.        ,
         0.99687964,  1.        , -1.        , -1.        ,  1.        ,
        -1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         0.99999976,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        , -1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        , -0.99999934,  1.        ]],
      dtype=float32), array([[-0.056637

1/1 [==============================] - 8s 8s/step - loss: 91.4280
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 90.6725
Epoch 1/1
1/1 [==============================] - 17s 17s/step - loss: 90.4483
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 90.4181
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 90.2221
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 89.4782
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 89.5090
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 89.3576
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 89.1995
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 89.1111
[array([[-0.9999013 ,  0.99999994,  1.        , -1.        ,  1.        ,
        -1.        , -1.        ,  1.        ,  1.        ,  1.        ,
        -1.        , -1.        ,  1.        , -1.        , -1.        ,
         1.        , -1.    

Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.3205
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.2911
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.1541
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.1380
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.1272
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 88.1006
[array([[ 1.        , -1.        , -1.        , -0.99616957,  1.        ,
        -1.        ,  1.        ,  1.        , -1.        ,  1.        ,
        -1.        ,  0.9406981 ,  1.        , -1.        ,  1.        ,
         1.        , -1.        , -1.        , -1.        ,  1.        ,
         1.        ,  1.        ,  1.        , -1.        , -1.        ,
         1.        , -0.9999999 ,  1.        ,  1.        ,  1.        ,
         0.9987465 ,  1.        ,  1.        ,  0.99999994,  1.        ,
         1.        , -1.        

Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 89.8480
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 89.3359
[array([[-1.        ,  1.        ,  1.        , -1.        ,  1.        ,
        -1.        , -1.        , -1.        , -1.        ,  0.9998882 ,
         1.        ,  1.        , -1.        , -1.        ,  1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
         0.9997465 ,  1.        ,  1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -0.98963124, -1.        ,
        -1.        ,  1.        ,  0.9933076 , -1.        ,  1.        ,
        -1.        ,  1.        , -1.        , -0.99999994,  1.        ,
        -1.        , -1.        ,  1.        ,  1.        , -1.        ,
        -1.        ,  1.        ,  1.        , -1.        , -1.        ]],
      dtype=float32), array([[0.11203539]], dtype=float32), array([[-0.8988512]], dtype=float32)]
Epoch 1/1
1/1 [==

KeyboardInterrupt: 